Name:   Anh Tuan Tran
Matrikelnummer:  7015463
Email:   antr00001@stud.uni-saarland.de
   
Name:   Deborah Dormah Kanubala
Matrikelnummer:   7025906
Email:  dkanubala@aimsammi.org

Name:    Irem Begüm Gündüz
Matrikelnummer:     7026821
Email: irgu00001@stud.uni-saarland.de

### Note: This assignment will extensively refer to coding exercise in assignment 4.

## 6.2.a Building your own Neural-Network

Import numpy, which is really all we need to create our own NN.

In [1]:
import numpy as np
np.random.seed(42)

# Remember to Fix your seeds for pytorch and numpy

Recall that our simple neural network consisted of two layers. We also added an `activation` function as a non-linearity to the output of our intermediate layer. Given an input $\mathbf{x} \in \mathbb{R}^n $ we have

$ \mathbf{h} = f^{(1)}(\mathbf{x}; \mathbf{W},c) = activation\_fn(\mathbf{W}^\mathsf{T} \mathbf{x} + c) $ 

$ \mathbf{y} = f^{(2)}(\mathbf{h}; \mathbf{w},b) = \text{$ softmax $}( \mathbf{w}^\mathsf{T} \mathbf{h} + b) $

In this exercise you will create your own network and are free to implement it with your own design choices. However, we will do it in a way that allows you to specify the depth of network, i.e. we extend our network such that there isn't just one $\mathbf{h}$ intermediate layers, but rather $n$ of them $\mathbf{h}_{i}$ with $i \in \{1,..., n\}$

**NOTE**: You are not allowed to use any built-in functions to calculate Leaky_ReLU, Softmax or the forward/backward pass directly.

**NOTE 2**: Remember to include the non-linearity at every layer. Remember to also add the bias to every layer. Finally, remember to apply the softmax in the output layer.

## ToDo: Rewrite the Leaky_ReLu and Softmax function as Class and implement a function in each of them to calculate gradients (1 point)
Remember that in PyTorch, these are implemented as classes so we also want to have them as classes.

In [15]:
class Leaky_Relu():
    """
    Recall your implementation of relu function in assignment 4 and try to implement
    Leaky_ReLu similarily, but as a class with a function to calculate gradient
    """
    # TODO: Implement
    def __init__(self, x):
        self.x = x
           
    def leaky_relu(self):
        relu = np.maximum(0.01*self.x, self.x)
        return relu
      
 
    def leaky_relu_grad(self):
        dx_relu = np.maximum(self.x, 1)
        return dx_relu
    

In [16]:
class Softmax():
    """
    Recall your implementation of softmax function in assignment 4 and try to implement
    softmax similarily, but as a class with a function to calculate gradient
    """
    # TODO: Implement
    def __init__(self, x):
        self.x = x
        
    def softmax(self):
        return np.exp(self.x)/ np.sum(np.exp(self.x), axis = 0)
    
    def softmax_grad(self):
        softmax_val = self.softmax()
        return softmax_val*(1-softmax_val)
        

## ToDo: Calculate the gradient using your implemented functions in their respective classes and validate by manually calculating gradients using a toy value. (1 point)

In [17]:
## Solution

## Leaky relu gradients:
val = 3.63
leaky_relu_val = Leaky_Relu(val)

##Softmax gradients
softmax_val = Softmax(val)

print("The leaky relu is {} and the softmax gradient is {}".format(leaky_relu_val.leaky_relu_grad(), 
                                                                  softmax_val.softmax_grad()))

The leaky relu is 3.63 and the softmax gradient is 0.0


## ToDo: Rewrite the code from Assignment 4 to include backpropagation in your class without using pytorch. Remember to use your Leaky_ReLu class here as activation function. (1.5 points)
#### Feel free to refer to your solutions from Assignment 4.

In [129]:
class FFNetwork:
    """
    Class representing the feed-forward neural network
    """
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, hidden_size: int):
        """
        Args:
        input_dim: dimensionality of `x`
        hidden_dim: dimensionality of the intermediate `h_i`
        output_dim: dimensionality of `y`
        hidden_size: number of intermediate layers `h_i`
        """
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.hidden_size = hidden_size
        
        self.linear_weights = []

        for i in range (hidden_size):
            if i == 0:
                inp_dim = input_dim
                out_dim = hidden_dim
            elif i != hidden_size -1:
                inp_dim = hidden_dim
                out_dim = hidden_dim
            else:
                inp_dim = hidden_dim
                out_dim = output_dim
            w = np.random.randn (inp_dim, hidden_dim)
            bias = np.random.randn (hidden_dim)
            self.linear_weights.append ({
                'w': w,
                'bias': bias,
            })
            w_2 = np.random.randn (hidden_dim, out_dim)
            bias = np.random.randn (out_dim)
            self.linear_weights.append ({
                'w': w_2,
                'bias': bias,
            })
        
    
    def forward(self, x):
        """
        Args:
        x: input to the neural network
        
        Output:
        `y`, i.e. the prediction of the network
        
        """
        res = x
        res = np.array(res)
        self.forward_z  = {}
        self.forward_h = {}
        print(self.hidden_size)
        for i in range (0,self.hidden_size+1):
            print(f"the iteration number {i}")
            res = res @ self.linear_weights[i]['w'] + self.linear_weights[i]['bias']
            self.forward_h[i] = res
            res = Leaky_Relu(res)
            res = res.leaky_relu()
            self.forward_z[i] = res
        res = Softmax(res)
        res = res.softmax()
        
        return res
    

  
    def backward_prop(self, x, y, res):
        lr = 0.0001
        loss = ((res - y)**2/len(x))
        print("the forward", self.forward_h)
        dx_soft_val1 = Softmax(self.forward_h[1])
        dx_soft_val1 = dx_soft_val1.softmax_grad()
        dw_2 = dx_soft_val1 *  self.forward_z[0]* loss
        dx_soft_val2 = Softmax(self.forward_h[1])
        dx_soft_val2 = dx_soft_val2.softmax_grad()
        dx_leaky = Leaky_Relu(self.forward_h[0])
        dx_leaky = dx_leaky.leaky_relu_grad()
        dw_1 = dx_soft_val2 *self.linear_weights[1]['w']* dx_leaky*x*loss
        self.linear_weights[0]['w'] = self.linear_weights[0]['w'] - lr * dw_1
        self.linear_weights[1]['w'] = self.linear_weights[1]['w'] - lr * dw_2
        return self.linear_weights


In [130]:

# forward prop of our network
network = FFNetwork(2, 2, 2, 1)
res = network.forward([1.,0.])

# backward prop of our network
network.backward_prop([12], 10, res)

1
the iteration number 0
the iteration number 1


### 6.2.b.2 Training a network for MNIST (1.5 points)

Now that we know how to train a Neural network in Pytorch. Let's start training and evaluating our model using a very standard dataset, for now let's use MNIST. Design a network from scracth using PyTorch and include the followings. Remember that we need to use forward-propagation and backprop.
- Training Loop
- Optimization 
- Evaluating Loop

In [32]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F


'''
LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

batch_size = 30



train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
CREATE MODEL CLASS WITH FORWARD AND BACKWARD
'''
class TorchFFNetwork(nn.Module):
    """
    A pytorch implementation to do classification for MNIST dataset.
    """
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, hidden_size: int):
        """
        Args:
        input_dim: dimensionality of `x`
        hidden_dim: dimensionality of the intermediate `h_i`
        output_dim: dimensionality of `y`
        hidden_size: number of intermediate layers `h_i`
        """
        ## SOLUTION ##
        super(TorchFFNetwork, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.hidden_size = hidden_size
        
        self.linear_layers = nn.ModuleList ([])
        for i in range (hidden_size):
            if i == 0:
                inp_dim = input_dim
                out_dim = hidden_dim
            elif i != hidden_size -1:
                inp_dim = hidden_dim
                out_dim = hidden_dim
            else:
                inp_dim = hidden_dim
                out_dim = output_dim
            self.linear_layers.append (nn.Linear(inp_dim, out_dim, bias=True))
        self.softmax = torch.nn.Softmax (dim=1)
        self.relu = torch.nn.ReLU()
        ## SOLUTION ##
    
    def forward(self, x):
        """
        Args:
        x: input to the neural network
        
        Output:
        `y`, i.e. the prediction of the network
        
        Note: Remember to apply the ReLU and add the bias for each layer
        """
    
        res = x.view(x.size(0), -1)
        for i in range (self.hidden_size):
            res = self.linear_layers[i](res)
            res = self.relu(res)

        res = self.softmax(res)
        
        return res
    

    


In [34]:
import numpy as np
# forward prop of our network
network = TorchFFNetwork(28*28, 2, 10, 2)
preds = network.forward(next(iter(train_loader))[0])


torch.Size([30, 10])

In [145]:
def train_loop(data, model, loss_fn, optimizer):  
    model.train ()
    num_epoch = 4
    for i_epoch in range (num_epoch):
        epoch_losses = []
        for batch in  iter(data):
            x, y = batch
            optimizer.zero_grad()
            y_pred = model(x)
            loss = loss_fn (y_pred, y)
            loss.backward()
            optimizer.step()
            epoch_losses.append (loss.detach ().numpy ())
#         if i_epoch % 100 == 0:
        print ("epoch:", i_epoch, "mean epoch loss:", np.mean (epoch_losses))

def evaluate_loop(data, model, loss_fn):
    model.eval()
    size = len(data)
    test_loss, correct = 0, 0
    i = 0
    with torch.no_grad():
        for batch in iter(data):
            
            X, y = batch
            i += int(y.size(0))
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).sum().item()
            
    print("old correct", correct)
    test_loss /= i
    correct /= i
    
    print("new correct", correct)
    print(f"Test Error : \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            

In [135]:
# Create the model from the forward class and pick a learning rate
torch.manual_seed(42)
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-2
(inp_dim, hidden_dim, out_dim, hidden_size) = (28*28, 2, 10, 2)
model = TorchFFNetwork(inp_dim, hidden_dim, out_dim, hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_loop(train_loader, model, loss_fn, optimizer)   


epoch: 0 mean epoch loss: 2.3018522
epoch: 1 mean epoch loss: 2.3018048
epoch: 2 mean epoch loss: 2.3017955
epoch: 3 mean epoch loss: 2.3017802


In [146]:
# evaluation of the model

evaluate_loop(test_loader, model, loss_fn)

old correct 1135
new correct 0.1135
Test Error : 
 Accuracy: 11.3%, Avg loss: 0.076877 



### ToDo: Implement functions for Stochastic Gradient Descent and Stochastic Gradient Descent with momentum and plot the difference on how they change the value for gradients. ( 1 + 1 (Bonus))